# Here we will use ptune learning method as baseline for genre classification


In [1]:
import sys
import os

sys.path.append(os.path.abspath('../'))

In [2]:
import numpy as np

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from transformers import AutoModelForCausalLM, AutoTokenizer

from src.utils import logger, DatasetTypes
from src.data import init_data
from src.ptune import prepare_ptune, train, MultiLabelClassifier
from src.model import get_pretrained
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import json
import re
# supported files in spython
device

/home/super-pc2/miniconda3/envs/genre-lyrics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [3]:
model_name = "Qwen/Qwen3-0.6B"
tokenizer, base_model = get_pretrained(model_name, device)

## Get dataset with all genres and 1,294,054 examples

In [4]:
path_to_csv = '../data/top_genres.csv'
train_dataset, val_dataset, test_dataset , idx2genre, genres, train_loader, val_loader, test_loader = init_data(path_to_csv=path_to_csv, batch_size=16, tokenizer=tokenizer)

In [5]:
peft_model = prepare_ptune(model=base_model, model_name=model_name, genres=genres, device=device)
hidden_size = base_model.config.hidden_size
num_labels = len(genres)
model = MultiLabelClassifier(peft_model, hidden_size, num_labels).to("cuda")

trainable params: 30,720 || all params: 596,080,640 || trainable%: 0.0052


In [6]:
num_epochs = 100
warmup_steps = 500
learning_rate = 5e-5

In [7]:
model = train(model=model,
      idx2genre=idx2genre,
      learning_rate=learning_rate,
      num_epochs=num_epochs,
      tokenizer=tokenizer,
      train_loader=train_loader,
      val_loader=val_loader,
      device=device)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch 1/100 - Train loss: 1.0756
Epoch 1/100 - Val macro F1: 0.1123, Label acc: 0.5350
Epoch 2/100 - Train loss: 1.0617
Epoch 2/100 - Val macro F1: 0.1206, Label acc: 0.5400
Epoch 3/100 - Train loss: 1.0317
Epoch 3/100 - Val macro F1: 0.1216, Label acc: 0.5525
Epoch 4/100 - Train loss: 0.9797
Epoch 4/100 - Val macro F1: 0.1155, Label acc: 0.5733
Epoch 5/100 - Train loss: 0.8948
Epoch 5/100 - Val macro F1: 0.1402, Label acc: 0.6100
Epoch 6/100 - Train loss: 0.8016
Epoch 6/100 - Val macro F1: 0.1409, Label acc: 0.6433
Epoch 7/100 - Train loss: 0.7209
Epoch 7/100 - Val macro F1: 0.1331, Label acc: 0.6575
Epoch 8/100 - Train loss: 0.6438
Epoch 8/100 - Val macro F1: 0.1353, Label acc: 0.6892
Epoch 9/100 - Train loss: 0.5605
Epoch 9/100 - Val macro F1: 0.1322, Label acc: 0.7283
Epoch 10/100 - Train loss: 0.4886
Epoch 10/100 - Val macro F1: 0.1313, Label acc: 0.7783
Epoch 11/100 - Train loss: 0.4416
Epoch 11/100 - Val macro F1: 0.1192, Label acc: 0.8033
Epoch 12/100 - Train loss: 0.4069
Epoch

/home/super-pc2/miniconda3/envs/genre-lyrics/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 36/100 - Val macro F1: 0.1486, Label acc: 0.8567
Epoch 37/100 - Train loss: 0.2692
Epoch 37/100 - Val macro F1: 0.1561, Label acc: 0.8508
Epoch 38/100 - Train loss: 0.2709


/home/super-pc2/miniconda3/envs/genre-lyrics/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


...

/home/super-pc2/miniconda3/envs/genre-lyrics/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 98/100 - Val macro F1: 0.1544, Label acc: 0.8608
Epoch 99/100 - Train loss: 0.2009


/home/super-pc2/miniconda3/envs/genre-lyrics/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 99/100 - Val macro F1: 0.1477, Label acc: 0.8575
Epoch 100/100 - Train loss: 0.2015
Epoch 100/100 - Val macro F1: 0.1553, Label acc: 0.8617


/home/super-pc2/miniconda3/envs/genre-lyrics/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [10]:
import numpy as np
from sklearn.metrics import f1_score

model.eval()

# Choose threshold range
thresholds = np.arange(0.1, 0.91, 0.05)

# To store best results
best_threshold = 0.0
best_f1 = 0.0
threshold_f1_scores = []

all_val_preds = []
all_val_labels = []

# Collect raw logits for threshold testing
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask)
        probs = torch.sigmoid(logits)

        all_val_preds.append(probs.cpu())     # save sigmoid probs
        all_val_labels.append(labels.cpu())

# Concatenate all batches
all_val_probs = torch.cat(all_val_preds, dim=0).to(torch.float32).numpy()
all_val_labels = torch.cat(all_val_labels, dim=0).to(torch.float32).numpy()


# Search for best threshold
for threshold in thresholds:
    preds = (all_val_probs > threshold).astype(int)
    f1 = f1_score(all_val_labels, preds, average='macro', zero_division=0)
    threshold_f1_scores.append((threshold, f1))

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

# Report
print(f"\nBest threshold: {best_threshold:.2f} with macro F1-score: {best_f1:.4f}")
print("Threshold sweep results:")
for t, f1 in threshold_f1_scores:
    print(f"Threshold = {t:.2f} --> Macro F1 = {f1:.4f}")



Best threshold: 0.15 with macro F1-score: 0.1911
Threshold sweep results:
Threshold = 0.10 --> Macro F1 = 0.1853
Threshold = 0.15 --> Macro F1 = 0.1911
Threshold = 0.20 --> Macro F1 = 0.1909
Threshold = 0.25 --> Macro F1 = 0.1836
Threshold = 0.30 --> Macro F1 = 0.1807
Threshold = 0.35 --> Macro F1 = 0.1689
Threshold = 0.40 --> Macro F1 = 0.1654
Threshold = 0.45 --> Macro F1 = 0.1590
Threshold = 0.50 --> Macro F1 = 0.1553
Threshold = 0.55 --> Macro F1 = 0.1432
Threshold = 0.60 --> Macro F1 = 0.1273
Threshold = 0.65 --> Macro F1 = 0.1265
Threshold = 0.70 --> Macro F1 = 0.1169
Threshold = 0.75 --> Macro F1 = 0.1179
Threshold = 0.80 --> Macro F1 = 0.0966
Threshold = 0.85 --> Macro F1 = 0.0858
Threshold = 0.90 --> Macro F1 = 0.0529
